In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob
import seaborn as sns
import numpy as np

## Read from `~/metrics/epoch_metrics/` directory

In [79]:
files = glob.glob("../../metrics/epoch_metrics/*parquet")

dataframes = []

for file in files:
    df = pd.read_parquet(file)
    
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

In [80]:
df["per_layer"] = df['total_rounds'].apply(lambda x: not isinstance(x, int))

In [81]:
import ast

def per_layer_columns(row):
    if row['per_layer']:
        
        tot_rounds = row['total_rounds']
        
        if isinstance(tot_rounds, str):
            return ast.literal_eval(row['total_rounds'])
        
        elif isinstance(row['total_rounds'], np.ndarray):
            return list(row['total_rounds'])
            
        else:
            print(type(row['total_rounds']))
            raise Exception("WTF")
    
    return row['total_rounds']

In [82]:
df['total_rounds'] = df.apply(per_layer_columns, axis=1)

In [83]:
#df = pd.read_parquet("../../metrics/epoch_metrics/")

## For Synchronous - "Centralized" investigation

In [84]:
df_synchronous = df[(df['fda_name'] == 'synchronous') & (df['aggr_scheme'] == 'avg')].copy()

In [85]:
df_synchronous['centralized_batch_size'] = df_synchronous['num_clients'] * df_synchronous['batch_size']

In [86]:
df_synchronous = df_synchronous.sort_values(by=['centralized_batch_size'], ascending=True)

# Hyper-Parameters

`bias` `NaN` -> no bias

`bias` $ \in (0, 1)$ -> `bias`% of each client's dataset is biased (from sorted sequence of MNIST) - rest is good

`bias` $= -1$ -> MNIST label 0 is sorted, placed at the start, and clients get assigned sequentially from this dataset. Only one label is biased to however many clients can get it assigned to them.

`bias` $= -2$ -> MNIST label 8 is sorted, placed at the start, and clients get assigned sequentially from this dataset. Only one label is biased to however many clients can get it assigned to them.

In [87]:
one_label_bias = {
    -1: 0,
    -2: 8
}

In [88]:
aggr = {
    'avg': '',
    'wavg_drifts': ' wavg'
}

In [89]:
per_layer_dict = {
    False: '',
    True: ' per-layer'
}

# Helpful new Dataframe metrics

### Add Helpful model metrics

In [90]:
df['model_bytes'] = df['nn_num_weights'] * 4

### Add Helpful FDA method metrics

In [91]:
def fda_local_state_bytes(row):
    if row['fda_name'] == "naive":
        return 4
    if row['fda_name'] == "linear":
        return 8
    if row['fda_name'] == "sketch":
        return row['sketch_width'] * row['sketch_depth'] * 4 + 4
    if row['fda_name'] in ["synchronous", "FedAdam", "FedAvgM"]:
        return 0
    if row['fda_name'] == 'gm':
        return 0.125  # one bit

In [92]:
df['local_state_bytes'] = df.apply(fda_local_state_bytes, axis=1)

### Add Total Steps

total steps (a single fda step might have many normal SGD steps, batch steps)

In [93]:
df['total_steps'] = df['total_fda_steps'] * df['num_steps_until_rtc_check']

### Add communication metrics

In [94]:
adv_layer_count = [640, 36928, 73856, 147584, 295168, 590080, 1180160, 262656, 5130]
lenet_layer_count = [156, 2416, 48120, 10164, 850]

The communication bytes exchanged for model synchronization. Remember that the Clients send their models to the Server and the Server sends the global model back. This happens at the end of every round.

In [95]:
# TODO: Densenet

def model_bytes_exchanged(row):
    if row['per_layer']:
        if row['nn_name'] == 'AdvancedCNN':
            return sum((layer_num_weights * 4) * layer_rounds * row['num_clients'] * 2 for layer_num_weights, layer_rounds in zip(adv_layer_count, row['total_rounds']))
        if row['nn_name'] == 'LeNet-5':
            print("lenet?")
            return sum((layer_num_weights * 4) * layer_rounds * row['num_clients'] * 2 for layer_num_weights, layer_rounds in zip(lenet_layer_count, row['total_rounds']))
    
    return row['total_rounds'] * row['model_bytes'] * row['num_clients'] * 2

In [96]:
df['model_bytes_exchanged'] = df.apply(model_bytes_exchanged, axis=1)

The communication bytes exchanged for monitoring the variance. This happens at the end of every FDA step which consists of `num_steps_until_rtc_check` number of steps. 

In [97]:
def monitoring_bytes_exchanged(row):
    if row['per_layer']:
        if row['nn_name'] == 'AdvancedCNN':
            return row['local_state_bytes'] * row['total_fda_steps'] * row['num_clients'] * len(adv_layer_count)
        if row['nn_name'] == 'LeNet-5':
            return row['local_state_bytes'] * row['total_fda_steps'] * row['num_clients'] * len(lenet_layer_count)
    
    return row['local_state_bytes'] * row['total_fda_steps'] * row['num_clients']

In [98]:
df['monitoring_bytes_exchanged'] = df.apply(monitoring_bytes_exchanged, axis=1)

The total communication bytes exchanged in the whole Federated Learning lifecycle.

In [99]:
df['total_communication_bytes'] = df['model_bytes_exchanged'] + df['monitoring_bytes_exchanged']

In [100]:
df['total_communication_gb'] = df['total_communication_bytes'] / 10**9

# HyperParameter ranking

### LeNet-5 - MNIST
On `Nvidia A10`:
1. Batch Size = 32 -> 6.613 ms ± 0.128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 7.509 ms ± 0.065 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 8.02 ms ± 0.099 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 9.258 ms ± 0.336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### AdvancedCNN - MNIST

On `Nvidia A10`:
1. *Batch Size* = 32 -> 8.853 ms ± 0.0917 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 10.325 ms ± 0.215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 11.989 ms ± 0.134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 16.47 ms ± 0.294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### DenseNet121 - CIFAR10

On `Nvidia A10`:
1. *Batch Size* = 32 -> 87.328 ms ± 0.9547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 88.832 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 94.208 ms ± 2.634 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 96.0 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### DenseNet201 - CIFAR10

On `Nvidia A10`:
1. *Batch Size* = 32 -> 142.0896 ms ± 2.9547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 150.2592 ms ± 5.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 154.5664 ms ± 6.758 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 156.5776 ms ± 7.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [101]:
step_ms = {
    ("AdvancedCNN", 32): 8.853,
    ("AdvancedCNN", 64): 10.325,
    ("AdvancedCNN", 128): 11.989,
    ("AdvancedCNN", 256): 16.47,
    ("LeNet-5", 32): 6.613,
    ("LeNet-5", 64): 7.509,
    ("LeNet-5", 128): 8.02,
    ("LeNet-5", 256): 9.258,
    ("DenseNet121", 32): 87.328,
    ("DenseNet121", 64): 88.832,
    ("DenseNet121", 128): 94.208,
    ("DenseNet121", 256): 96.0,
    ("DenseNet201", 32): 142.0896,
    ("DenseNet201", 64): 150.2592,
    ("DenseNet201", 128): 154.5664,
    ("DenseNet201", 256): 156.5776
}

In [102]:
import numpy as np

def cpu_time_cost(row):
    """ Total cpu time cost in (sec).
    A single `step` means each client performed a single `step` 
    """
    return row['total_steps'] * step_ms[(row['nn_name'], row['batch_size'])] / 1000

def communication_time_cost(num_clients, total_communication_bytes, comm_model):
    """ Assuming channel is 1Gbps """

    total_communication_gbit = total_communication_bytes * 8e-9

    if comm_model == 'common_channel':
        
        return ((num_clients - 1) / num_clients) * total_communication_gbit    # sec

    if comm_model == 'hypercube':

        return (np.ceil(np.log(num_clients)) / num_clients) * total_communication_gbit   # sec

In [103]:
df['cpu_time_cost'] = df.apply(cpu_time_cost, axis=1)

In [104]:
df['hypercube_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'hypercube')

In [105]:
df['common_channel_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'common_channel')

In [106]:
df['hypercube_time_cost'] = df['cpu_time_cost'] + df['hypercube_communication_time_cost']

In [107]:
df['common_channel_time_cost'] = df['cpu_time_cost'] + df['common_channel_communication_time_cost']

In [108]:
df['hypercube_comm_cpu_time_ratio'] = df['hypercube_communication_time_cost'] / df['cpu_time_cost']

In [109]:
df['common_channel_comm_cpu_time_ratio'] = df['common_channel_communication_time_cost'] / df['cpu_time_cost']

# Plots about cost

In [110]:
# Define styles for each fda_name
# TODO
fda_styles = {
    'naive': 'o-r',
    'linear': 's-g',
    'sketch': '^-b',
    'synchronous': 'x-c'
}
fda_names = ['gm', 'naive', 'linear', 'sketch', 'FedAvgM', 'FedAdam', 'synchronous']

In [111]:
import matplotlib

num_clients_values = sorted(df['num_clients'].unique())
cmap = matplotlib.colormaps['tab20b']
colors_dict = {
    num_clients: color 
    for num_clients, color in zip(num_clients_values, cmap(np.linspace(0, 1, len(num_clients_values))))
}

## KDE Helper

In [112]:
"""
sns_params = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': True,
    'alpha': 0.2
}
"""
sns_params = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': False,
    'alpha': 1
}

sns_params_biases = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': False,
    'alpha': 0.8
}

base_colors = {
    'gm': 'black',
    'naive': 'orange',
    'linear': 'green',
    'sketch': 'red',
    'synchronous': 'purple',
    'sketch wavg': 'black',
    'FedAvgM': 'blue',
    'FedAdam': 'blue'
}

base_colors_per_layer = {
    'gm': 'black',
    'naive': 'orange',
    'linear': 'green',
    'sketch': 'red',
    'synchronous': 'purple',
    'naive per-layer': 'black',
    'FedAvgM': 'blue',
    'FedAdam': 'blue'
}

sync_colors = {
    32: 'rebeccapurple',
    64: 'darkorchid',
    128: 'mediumorchid',
    256: 'plum'
}

In [113]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 14

## Total time cost with accuracy

### KDE

In [114]:
def kde_time_cost(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        
        for per_layer in per_layers:
                    
            per_layer_df = df_bias[df_bias['per_layer'] == per_layer]
        
            for fda_name in fda_names:
                
                name = f"{fda_name}{per_layer_dict[per_layer]}"
                
                fda_df = per_layer_df[per_layer_df['fda_name'] == fda_name]
                
                if fda_df.empty:
                    continue

                common_channel_df = fda_df['common_channel_time_cost']
                sns.kdeplot(common_channel_df, label=name, ax=axs[0], color=base_colors_per_layer[name], log_scale=log_scale, **sns_params)

                hypercube_df = fda_df['hypercube_time_cost']
                sns.kdeplot(hypercube_df, label=name, ax=axs[1], color=base_colors_per_layer[name], log_scale=log_scale, **sns_params)


        if not x_log:
            axs[0].set_xlim(left=0)
        axs[0].set_xlabel('Time Cost (sec)')
        axs[0].set_ylabel('Density')
        axs[0].legend()
        axs[0].set_title("Common Channel Communication Model")
        axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[0].legend()

        if not x_log:
            axs[1].set_xlim(left=0)

        axs[1].set_xlabel('Time Cost (sec)')
        axs[1].set_ylabel('Density')
        axs[1].legend()
        axs[1].set_title("Hypercube Communication Model")
        axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[1].legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        fig.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(fig)

        plt.close(fig)
    
        
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

## Total Communication cost (in gb) with accuracy (scatter)

### KDE

In [115]:
def kde_communication_cost(df, filename, x_log=True):

    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        plt.figure(figsize=(10, 6))
        avg_info = []
        
        for per_layer in per_layers:
                    
            per_layer_df = df_bias[df_bias['per_layer'] == per_layer]
        
            for fda_name in fda_names:
                
                name = f"{fda_name}{per_layer_dict[per_layer]}"
                
                fda_df = per_layer_df[per_layer_df['fda_name'] == fda_name]
                
                if fda_df.empty:
                    continue

                fda_data_df = fda_df['total_communication_gb']

                avg_info.append(f'{name}: {fda_data_df.mean():.2f} GB')

                # Plotting only the KDE using kdeplot
                sns.kdeplot(fda_data_df, label=name, log_scale=log_scale, color=base_colors_per_layer[name], **sns_params)

        text = "Average Communication:\n" + '\n'.join(avg_info)
        # Add the text annotation inside the plot
        plt.text(0.02, 0.97, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))

        if not x_log:
            plt.xlim(left=0)

        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        plt.xlabel('Communication (GB)')
        plt.ylabel('Density')
        #plt.legend()  # We'll be moving this
        plt.gca().legend(loc='best', bbox_to_anchor=(0, 0.05, 1, 0.67))
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        plt.suptitle(info+bias_title)

        plt.tight_layout()

        pdf.savefig(plt.gcf()) # Save the current figure

        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
    pdf.close()

## Total CPU time (in Seconds) with accuracy

### KDE

In [116]:
def kde_cpu_time_cost(df, filename, x_log=False):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        plt.figure(figsize=(10, 6))
        
        for per_layer in per_layers:
                    
            per_layer_df = df_bias[df_bias['per_layer'] == per_layer]
        
            for fda_name in fda_names:
                
                name = f"{fda_name}{per_layer_dict[per_layer]}"
                
                fda_df = per_layer_df[per_layer_df['fda_name'] == fda_name]
                
                if fda_df.empty:
                    continue

                fda_data_df = fda_df['cpu_time_cost']

                # Plotting only the KDE using kdeplot
                sns.kdeplot(fda_data_df, label=name, log_scale=log_scale, color=base_colors_per_layer[name], **sns_params)

        if not x_log:
            plt.xlim(left=0)
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        plt.xlabel('CPU time cost (sec)')
        plt.ylabel('Density')
        plt.legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        plt.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(plt.gcf()) # Save the current figure
        
        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
    pdf.close()

## Total CPU cost / Comm Cost (time) - ratio 

In [117]:
def kde_comm_cpu_time_cost_ratio(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        
        for per_layer in per_layers:
                    
            per_layer_df = df_bias[df_bias['per_layer'] == per_layer]
        
            for fda_name in fda_names:
                
                name = f"{fda_name}{per_layer_dict[per_layer]}"
                
                fda_df = per_layer_df[per_layer_df['fda_name'] == fda_name]
                
                if fda_df.empty:
                    continue
            
                common_channel_ratio_df = fda_df['cpu_time_cost'] / fda_df['common_channel_communication_time_cost']
                sns.kdeplot(common_channel_ratio_df, label=name, ax=axs[0], color=base_colors_per_layer[name], log_scale=log_scale, **sns_params)

                hypercube_ratio_df = fda_df['cpu_time_cost'] / fda_df['hypercube_communication_time_cost']
                sns.kdeplot(hypercube_ratio_df, label=name, ax=axs[1], color=base_colors_per_layer[name], log_scale=log_scale, **sns_params)


        if not x_log:
            axs[0].set_xlim(left=0)
        axs[0].set_xlabel('CPU Time Cost / Comm. Time Cost')
        axs[0].set_ylabel('Density')
        axs[0].legend()
        axs[0].set_title("Common Channel Communication Model")
        axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[0].legend()

        if not x_log:
            axs[1].set_xlim(left=0)

        axs[1].set_xlabel('CPU Time Cost / Comm. Time Cost')
        axs[1].set_ylabel('Density')
        axs[1].legend()
        axs[1].set_title("Hypercube Communication Model")
        axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[1].legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        fig.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(fig)

        plt.close(fig)
    
        
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

## Biparte

In [118]:
sns_params_joint = {
    'bw_method': 'scott',
    'bw_adjust': 0.5,
    'fill': False,
    'alpha': 0.4, # 0.8
    'n_levels': 4,
    'fill': True,
}

sns_params_joint_tmp = {
    'bw_method': 'scott',
    'bw_adjust': 0.5,
    'fill': False,
    'alpha': 0.8,
    'n_levels': 4
}


def kde_joint(df, filename, x_log=True, y_log=False):

    log_scale = (x_log, y_log)
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if df_bias.empty or len(df_bias['fda_name'].unique()) < 2:
            continue
    
        plt.figure(figsize=(8, 8))
        
        if (len(df_bias[df_bias.fda_name == 'linear']) == 1 or df_bias[df_bias.fda_name == 'linear'].empty or
            len(df_bias[df_bias.fda_name == 'sketch']) == 1 or df_bias[df_bias.fda_name == 'sketch'].empty or
            len(df_bias[df_bias.fda_name == 'naive']) == 1 or df_bias[df_bias.fda_name == 'naive'].empty or
            len(df_bias[df_bias.fda_name == 'synchronous']) == 1) or df_bias[df_bias.fda_name == 'synchronous'].empty:
            continue

        # Plotting only the KDE using kdeplot
        """
        g = sns.jointplot(df_bias, x='total_communication_gb', y='cpu_time_cost', hue='fda_name', log_scale=log_scale, kind="kde", height=9, palette=base_colors, **sns_params_joint)

        if not x_log:
            plt.xlim(left=0)
        
        if not y_log:
            plt.ylim(bottom=0)
            
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        
        g.set_axis_labels("Communication Cost (GB)", "CPU Time Cost (sec)")

        g.ax_joint.get_legend().set_title(None)
        """

        g = sns.kdeplot(df_bias, x='total_communication_gb', y='cpu_time_cost', hue='fda_name', log_scale=log_scale, palette=base_colors, **sns_params_joint_tmp)
        g = sns.kdeplot(df_bias, x='total_communication_gb', y='cpu_time_cost', hue='fda_name', log_scale=log_scale, palette=base_colors, **sns_params_joint)
        
        # Set x and y limits if not log scale
        if not x_log:
            plt.xlim(left=0)

        if not y_log:
            plt.ylim(bottom=0)

        # Add grid
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

        # Set axis labels
        g.set_xlabel("Communication Cost (GB)")
        g.set_ylabel("CPU Time Cost (sec)")

        # Modify the legend
        legend = g.get_legend()
        
        if legend:
            legend.set_title(None)

        
        fda_df = df_bias[~df_bias.fda_name.isin(['synchronous', 'FedAdam', 'FedAvgM'])]
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        
        general = f"All strategies : {bias_title}  ,  bs : 32\n"
        fda_info = f"FDA :  Θ {sorted(list(fda_df['theta'].unique()))}"
        
        plt.suptitle(general+fda_info)

        plt.tight_layout()

        pdf.savefig(plt.gcf()) # Save the current figure

        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
    pdf.close()
    
    
def kde_joint_per_clients(df, filename, x_log=True, y_log=False):

    log_scale = (x_log, y_log)
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    num_clients_values = sorted(df['num_clients'].unique())

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if df_bias.empty or len(df_bias['fda_name'].unique()) < 2:
            continue
            
        for num_clients in num_clients_values:
            
            df_clients = df_bias[df_bias.num_clients == num_clients]
            
            if (len(df_clients[df_clients.fda_name == 'linear']) == 1 or df_clients[df_clients.fda_name == 'linear'].empty or
                len(df_clients[df_clients.fda_name == 'sketch']) == 1 or df_clients[df_clients.fda_name == 'sketch'].empty or
                len(df_clients[df_clients.fda_name == 'naive']) == 1 or df_clients[df_clients.fda_name == 'naive'].empty or
                len(df_clients[df_clients.fda_name == 'synchronous']) == 1) or df_clients[df_clients.fda_name == 'synchronous'].empty:
                continue
                                                             
            #plt.figure(figsize=(10, 6))
            
            if df_clients.empty:
                continue

            # Plotting only the KDE using kdeplot
            g = sns.jointplot(df_clients, x='total_communication_gb', y='cpu_time_cost', hue='fda_name', log_scale=log_scale, kind="kde", height=9, palette=base_colors, **sns_params_joint)

            if not x_log:
                plt.xlim(left=0)

            if not y_log:
                plt.ylim(bottom=0)

            plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

            g.set_axis_labels("Communication Cost (GB)", "CPU Time Cost (sec)")

            g.ax_joint.get_legend().set_title(None)


            fda_df = df_bias[~df_bias.fda_name.isin(['synchronous', 'FedAdam', 'FedAvgM'])]
            sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
            
            general = f"All strategies : {bias_title}  ,  Num. Clients: {num_clients}  ,  bs : 32\n"
            fda_info = f"FDA :  Θ {sorted(list(fda_df['theta'].unique()))}"
        
            plt.suptitle(general+fda_info)

            plt.tight_layout()

            pdf.savefig(plt.gcf()) # Save the current figure

            # Close the current figure to prevent it from being displayed in the notebook
            plt.close()
            
    pdf.close()

## Keep Hyper-parameters fixed and plot for filtered

In [119]:
def fda_methods_clients_time_cost(df, filename, without_synchronous=False):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]
    
    min_clients = min(df['num_clients'])
    max_clients = max(df['num_clients'])

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 

                if filtered_df.empty:
                    continue

                fig, axs = plt.subplots(1, 2, figsize=(20, 6))
                
                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        axs[0].plot(fda_data['num_clients'], fda_data['common_channel_time_cost'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)
                        axs[1].plot(fda_data['num_clients'], fda_data['hypercube_time_cost'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)

                if not without_synchronous:
                    sync_df = df_bias[df_bias['fda_name'] == 'synchronous'] 
                    sync_batch_sizes = sorted(sync_df['batch_size'].unique())

                    for sync_batch_size in sync_batch_sizes:
                        synchronous_data = sync_df[sync_df['batch_size'] == sync_batch_size] 

                        if not synchronous_data.empty:
                            axs[0].plot(synchronous_data['num_clients'], synchronous_data['common_channel_time_cost'], marker='o', label=f'synchronous b{sync_batch_size}', color=sync_colors[sync_batch_size], markersize=3)
                            axs[1].plot(synchronous_data['num_clients'], synchronous_data['hypercube_time_cost'], marker='o', label=f'synchronous b{sync_batch_size}', color=sync_colors[sync_batch_size], markersize=3)

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                if not without_synchronous: axs[0].set_yscale('log')
                axs[0].set_xticks(x_ticks)
                axs[0].set_xlabel('Number of clients')
                axs[0].legend()
                axs[0].set_title("Common Channel Communication Model")
                axs[0].set_ylabel('Time Cost')
                axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                if not without_synchronous: axs[1].set_yscale('log')
                axs[1].set_xticks(x_ticks)
                axs[1].set_xlabel('Number of clients')
                axs[1].legend()
                axs[1].set_title("Hypercube Communication Model")
                axs[1].set_ylabel('Time Cost')
                axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                fig.suptitle(title)

                plt.tight_layout()

                pdf.savefig(fig)

                plt.close(fig)
    pdf.close()

In [120]:
def fda_methods_clients_comm_cost(df, filename, without_synchronous=False):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    min_clients = min(df['num_clients'])
    max_clients = max(df['num_clients'])
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 

                if filtered_df.empty:
                    continue

                plt.figure(figsize=(10, 6))
                
                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['num_clients'], fda_data['total_communication_gb'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)

                
                if not without_synchronous:
                    sync_df = df_bias[df_bias['fda_name'] == 'synchronous'] 
                    sync_batch_sizes = sorted(sync_df['batch_size'].unique())

                    for sync_batch_size in sync_batch_sizes:
                        synchronous_data = sync_df[sync_df['batch_size'] == sync_batch_size] 

                        if not synchronous_data.empty:
                            plt.plot(synchronous_data['num_clients'], synchronous_data['total_communication_gb'], marker='o', label=f'synchronous b{sync_batch_size}', color=sync_colors[sync_batch_size], markersize=3)

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                if not without_synchronous:
                    plt.yscale('log')
                plt.xticks(x_ticks)
                plt.xlabel('Number of clients')
                plt.legend()
                plt.ylabel('Communication (GB)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf()) # Save the current figure
        
                # Close the current figure to prevent it from being displayed in the notebook
                plt.close()
    pdf.close()

In [121]:
def fda_methods_clients_cpu_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]
    
    min_clients = min(df['num_clients'])
    max_clients = max(df['num_clients'])

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 
                    
                if filtered_df.empty:
                    continue

                plt.figure(figsize=(10, 6))
                
                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['num_clients'], fda_data['cpu_time_cost'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)

                sync_df = df_bias[df_bias['fda_name'] == 'synchronous'] 
                sync_batch_sizes = sorted(sync_df['batch_size'].unique())

                for sync_batch_size in sync_batch_sizes:
                    synchronous_data = sync_df[sync_df['batch_size'] == sync_batch_size] 

                    if not synchronous_data.empty:
                        plt.plot(synchronous_data['num_clients'], synchronous_data['cpu_time_cost'], marker='o', label=f'synchronous b{sync_batch_size}', color=sync_colors[sync_batch_size], markersize=3)

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                #plt.yscale('log')
                plt.xticks(x_ticks)
                plt.xlabel('Number of clients')
                plt.legend()
                plt.ylabel('CPU Time Cost (sec)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf()) # Save the current figure
        
                # Close the current figure to prevent it from being displayed in the notebook
                plt.close()
    pdf.close()

In [122]:
def fda_methods_theta_time_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                fig, axs = plt.subplots(1, 2, figsize=(20, 6))

                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        axs[0].plot(fda_data['theta'], fda_data['common_channel_time_cost'], marker='o', label=name, color=base_colors_per_layer[name], markersize=3)
                        axs[1].plot(fda_data['theta'], fda_data['hypercube_time_cost'], marker='o', label=name, color=base_colors_per_layer[name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    axs[0].axhline(y=synchronous_data['common_channel_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)
                    axs[1].axhline(y=synchronous_data['hypercube_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                if not axs[0].has_data():
                    plt.close(fig)
                    continue

                axs[0].set_yscale('log')
                axs[0].set_xlabel('Theta')
                axs[0].legend()
                axs[0].set_title("Common Channel Communication Model")
                axs[0].set_ylabel('Time Cost')
                axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                axs[1].set_yscale('log')
                axs[1].set_xlabel('Theta')
                axs[1].legend()
                axs[1].set_title("Hypercube Communication Model")
                axs[1].set_ylabel('Time Cost')
                axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                fig.suptitle(title)

                plt.tight_layout()

                pdf.savefig(fig)

                plt.close(fig)
    pdf.close()

In [123]:
def fda_methods_theta_comm_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                plt.figure(figsize=(10, 6))

                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['theta'], fda_data['total_communication_gb'], marker='o', label=name, color=base_colors_per_layer[name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    plt.axhline(y=synchronous_data['total_communication_gb'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                plt.yscale('log')
                plt.xlabel('Theta')
                plt.legend()
                plt.ylabel('Communication Cost (GB)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf())

                plt.close()
    pdf.close()

In [124]:
def fda_methods_theta_cpu_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                plt.figure(figsize=(10, 6))

                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]

                    for fda_name in fda_names:
                        
                        if fda_name == 'synchronous':
                            continue

                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['theta'], fda_data['cpu_time_cost'], marker='o', label=name, color=base_colors_per_layer[name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    plt.axhline(y=synchronous_data['cpu_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                #plt.yscale('log')
                plt.xlabel('Theta')
                plt.legend()
                plt.ylabel('CPU Time Cost (sec)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf())

                plt.close()
    pdf.close()

# Per accuracy

In [125]:
def accuracies_plots(df, filename, accuracies):
    pdf = PdfPages(filename)

    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    per_layers = [False, True]

    for bias in biases:

        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'

        df_bias = df[mask]
        
        for bs in sorted(df_bias['batch_size'].unique()):

            bs_df = df_bias[df_bias['batch_size'] == bs]
                
            for theta in sorted(bs_df['theta'].unique())[1:]:

                theta_df = bs_df[bs_df['theta'] == theta]

                for client in sorted(theta_df['num_clients'].unique()):

                    client_df = theta_df[theta_df['num_clients'] == client]
                    
                    plt.figure(figsize=(10, 6))
                    
                    for per_layer in per_layers:
                    
                        per_layer_df = client_df[client_df['per_layer'] == per_layer]

                        for fda_name in fda_names:

                            if fda_name == 'synchronous':
                                continue

                            name = f"{fda_name}{per_layer_dict[per_layer]}"

                            fda_df = per_layer_df[per_layer_df['fda_name'] == fda_name]

                            if fda_df.empty:
                                continue

                            dataframes_for_acc = []
                            have = False

                            for acc in accuracies:

                                filtered_df = fda_df[(fda_df.accuracy > acc)]

                                if filtered_df.empty:
                                    continue

                                have = True

                                idx = filtered_df['epoch'].idxmin()
                                filtered_df = filtered_df.loc[idx]

                                dataframes_for_acc.append(filtered_df)

                            if not have:
                                continue

                            df_concat = pd.concat(dataframes_for_acc)

                            plt.plot(df_concat['accuracy'], df_concat['total_communication_gb'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)

                        synchronous_df = df_bias[(df_bias['num_clients'] == client) & (df_bias['fda_name'] == 'synchronous')]

                    for b in sorted(synchronous_df['batch_size'].unique()):

                        sync_df = synchronous_df[synchronous_df['batch_size'] == b]

                        if sync_df.empty:
                            continue

                        dataframes_for_acc = []

                        have = False

                        for acc in accuracies:

                            filtered_df = sync_df[(sync_df.accuracy > acc)]

                            if filtered_df.empty:
                                continue

                            have = True

                            idx = filtered_df['epoch'].idxmin()
                            filtered_df = filtered_df.loc[idx]

                            dataframes_for_acc.append(filtered_df)


                        if not have:
                            continue 

                        df_concat = pd.concat(dataframes_for_acc)

                        plt.plot(df_concat['accuracy'], df_concat['total_communication_gb'], color=sync_colors[b], marker='o', label=f'synchronous b{b}', markersize=3)

                    plt.yscale('log')
                    plt.xlabel('Accuracy')
                    plt.legend()
                    plt.ylabel('Communication Cost (GB)')
                    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                    title = f'$Theta$ : {theta} , Batch Size: {bs} , {bias_title} , Num Clients: {client}'
                    plt.title(title)

                    pdf.savefig(plt.gcf())

                    plt.close()
    pdf.close()

In [126]:
def centralized_synchronous_plots(df, acc_threshold, filename):
    pdf = PdfPages(filename)

    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    acc_df = df[(df.accuracy > acc_threshold)]
    
    # 2. Filter out same runs. We choose the instance which first hits the `acc_threshold`
    idx = acc_df.groupby(['centralized_batch_size', 'bias'], dropna=False)['epoch'].idxmin()
    filtered_acc_df = acc_df.loc[idx]

    for bias in biases:

        if pd.isna(bias):
            mask = filtered_acc_df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = filtered_acc_df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = filtered_acc_df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'

        df_bias = filtered_acc_df[mask]
        
        # ````````` here
        
        plt.figure(figsize=(10, 6))
        
        plt.plot(df_bias['centralized_batch_size'], df_bias['total_fda_steps'], marker='o', markersize=3)
        

        #plt.xscale('log')
        plt.ylabel('Steps')
        plt.xlabel('Batch Size')
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

        title = f'{bias_title}'
        plt.title(title)
        plt.suptitle("Synchronous Centralized")

        pdf.savefig(plt.gcf())

        plt.close()
        
    pdf.close()

# Help-Stat

In [127]:
def explore_top(df, acc_thresh, nn_name, fda_name):
    acceptable_acc_df = df[(df.accuracy > acc_thresh) & (df.nn_name == nn_name)]
    fda_df = acceptable_acc_df[acceptable_acc_df['fda_name'] == fda_name]
    
    idx = fda_df.groupby(['fda_name', 'num_clients', 'bias', 'batch_size', 'theta', 'aggr_scheme', 'per_layer'], dropna=False)['epoch'].idxmin()
    aggr_df = fda_df.loc[idx]
    
    print(len(aggr_df))
    
    aggr_df['monitoring_gb_exchanged'] = aggr_df['monitoring_bytes_exchanged'] / 10**9
    aggr_df['model_gb_exchanged'] = aggr_df['model_bytes_exchanged'] / 10**9
    
    #return filtered_acceptable_acc_df
    return aggr_df[['num_clients', 'batch_size', 'theta', 'bias', 'per_layer', 'total_rounds', 'total_fda_steps', 'epoch', 'monitoring_gb_exchanged', 'model_gb_exchanged', 'total_communication_gb', 'cpu_time_cost', 'hypercube_time_cost', 'common_channel_time_cost']].sort_values(by='total_communication_gb')

## Save all those time-cost plots

In [128]:
import os

def time_cost_plots(df, acc_threshold, nn_name, limit_x_axis=False, show_runs=False, params=False, kde_time_log=True, kde_comm_log=True, kde_cpu_log=False):
    acceptable_acc_df = df[(df.accuracy > acc_threshold)]
    
    str_thresh = str(acc_threshold).replace('.', '_')  # replace '.'
    
    if not os.path.exists(f"../../metrics/plots/{nn_name}/{str_thresh}"):
        os.makedirs(f"../../metrics/plots/{nn_name}/{str_thresh}")
        
    # 1. Same runs not included|
    
    # 2. Filter out same runs. We choose the instance which first hits the `acc_threshold`
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias', 'aggr_scheme', 'per_layer'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    #kde_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/time_cost.pdf", x_log=kde_time_log)
    
    #kde_communication_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/comm_cost.pdf", x_log=kde_comm_log)
    
    #kde_cpu_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/cpu_cost.pdf", x_log=kde_cpu_log)
    
    #kde_comm_cpu_time_cost_ratio(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/ratio.pdf", x_log=kde_comm_log)
    
    df_filt_simple = filtered_acceptable_acc_df[(~filtered_acceptable_acc_df.per_layer) & (filtered_acceptable_acc_df.aggr_scheme == 'avg') & (filtered_acceptable_acc_df.bias != 0.9)]  #  filter out weird tests
    
    #y_log = True if nn_name == 'DenseNet121' else False
    y_log = True
    
    kde_joint(df_filt_simple, f"../../metrics/plots/{nn_name}/{str_thresh}/joint_costs.pdf", y_log=y_log)
    #kde_joint_per_clients(df_filt_simple, f"../../metrics/plots/{nn_name}/{str_thresh}/joint_per_clients.pdf", y_log=y_log)
    
    # Parameters fixed, and plot
    if params:
        fda_methods_clients_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_time_cost.pdf")
        fda_methods_clients_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_comm_cost.pdf")
        fda_methods_clients_cpu_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_cpu_cost.pdf")
        
        fda_methods_clients_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_comm_cost_no_sync.pdf", without_synchronous=True)
        fda_methods_clients_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_time_cost_no_sync.pdf", without_synchronous=True)
        
        #fda_methods_theta_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_time_cost.pdf")
        #fda_methods_theta_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_comm_cost.pdf")
        #fda_methods_theta_cpu_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_cpu_cost.pdf")

In [129]:
df_32 = df[(df['batch_size'] == 32) & (df['aggr_scheme'] == 'avg')]

In [130]:
df_32 = df_32[df_32['fda_name'] != 'gm']

## DenseNet121 - CIFAR10

In [168]:
df_dense = df_32[((df_32.theta >= 250.0) | (df_32.theta == 0)) & (df_32['nn_name'] == 'DenseNet121')]

In [169]:
time_cost_plots(df_dense, 0.75, 'DenseNet121', params=False)
time_cost_plots(df_dense, 0.8, 'DenseNet121', params=False)

In [153]:
#accuracies_plots(df_dense, f"../../metrics/plots/DenseNet121/accuracies.pdf", accuracies=[0.3, 0.4, 0.5, 0.6, 0.7, 0.73, 0.76, 0.8, 0.81])

In [154]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'DenseNet121'], 0.79, f"../../metrics/plots/DenseNet121/0_8/centralized_synchronous.pdf")

In [155]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'DenseNet121'], 0.8, f"../../metrics/plots/DenseNet121/0_8/centralized_synchronous.pdf")

In [156]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'DenseNet121'], 0.81, f"../../metrics/plots/DenseNet121/0_81/centralized_synchronous.pdf")

## DenseNet201 - CIFAR10

In [157]:
df_dense = df_32[((df_32.theta >= 250.0) | (df_32.theta == 0)) & (df_32['nn_name'] == 'DenseNet201')]

In [158]:
time_cost_plots(df_dense, 0.79, 'DenseNet201', params=False)
time_cost_plots(df_dense, 0.8, 'DenseNet201', params=False)

## AdvancedCNN - MNIST

In [172]:
df_adv = df_32[((df_32.theta >= 20.0) | (df_32.theta == 0)) & (df_32.nn_name == 'AdvancedCNN')]

In [173]:
time_cost_plots(df_adv, 0.99, 'AdvancedCNN', params=False)
time_cost_plots(df_adv, 0.995, 'AdvancedCNN', params=False)

In [161]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'AdvncedCNN'], 0.99, f"../../metrics/plots/AdvancedCNN/0_99/centralized_synchronous.pdf")

In [162]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'AdvncedCNN'], 0.993, f"../../metrics/plots/AdvancedCNN/0_993/centralized_synchronous.pdf")

In [163]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'AdvncedCNN'], 0.995, f"../../metrics/plots/AdvancedCNN/0_995/centralized_synchronous.pdf")

In [164]:
#explore_top(df_lenet, 0.98, 'LeNet-5', 'synchronous')

## LeNet-5 - MNIST

In [165]:
df_lenet = df_32[((df_32.theta >= 3.0) | (df_32.theta == 0)) & (df_32.nn_name == 'LeNet-5')]

In [166]:
#explore_top(df, 0.98, 'LeNet-5', 'linear')

In [167]:
time_cost_plots(df_lenet, 0.98, 'LeNet-5', params=False)
time_cost_plots(df_lenet, 0.975, 'LeNet-5', params=False)
time_cost_plots(df_lenet, 0.985, 'LeNet-5', params=False)

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

In [87]:
#accuracies_plots(df_lenet[df_lenet['nn_name'] == 'LeNet-5'], f"../../metrics/plots/LeNet-5/accuracies_clients.pdf", accuracies=[0.96, 0.965, 0.96, 0.965, 0.98, 0.982, 0.984, 0.985, 0.987, 0.989])

In [73]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'LeNet-5'], 0.975, f"../../metrics/plots/LeNet-5/0_975/centralized_synchronous.pdf")

In [74]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'LeNet-5'], 0.98, f"../../metrics/plots/LeNet-5/0_98/centralized_synchronous.pdf")

In [75]:
#centralized_synchronous_plots(df_synchronous[df_synchronous['nn_name'] == 'LeNet-5'], 0.985, f"../../metrics/plots/LeNet-5/0_985/centralized_synchronous.pdf")